## Drazin Inverse Lab

### Problem 1:

In [82]:
import numpy as np
from cmath import sqrt
from scipy import linalg as la
from matplotlib import pyplot as plt
import pandas as pd

In [23]:
def is_drazin(A, k, Ad):
    if not np.allclose(A@Ad, Ad@A):
        return False
    if not np.allclose(np.linalg.matrix_power(A,k+1)@Ad, np.linalg.matrix_power(A,k)):
        return False
    if not np.allclose(Ad@A@Ad, Ad):
        return False
    return True

In [24]:
# Test cases

A = [[1, 3, 0, 0],
     [0, 1, 3, 0],
     [0, 0, 1, 3],
     [0, 0, 0, 0,]]

Ad = [[1, -3, 9, 81],
      [0, 1, -3, -18],
      [0, 0, 1, 3],
      [0, 0, 0, 0,]]

ka = 1

B  = [[1,1, 3],
      [5, 2, 6],
      [-2, -1, -3]]
A, Ad, B = map(np.asarray, (A, Ad, B))
Bd = np.zeros_like(B)
kb = 3

is_drazin(A, ka, Ad) # returns True
is_drazin(B, kb, Bd) #returns True

True

### Problem 2:

In [45]:
def drazin(A, tol=1e-5):
    n, n = np.shape(A)
    f = lambda x: abs(x) > tol
    g = lambda x: abs(x) <= tol
    Q1, S, k1 = la.schur(A, sort=f)
    Q2, T, k2 = la.schur(A, sort=g)
    U = np.column_stack((S[:,:k1], T[:,:(n-k1)]))
    U_inv = np.linalg.inv(U)
    V = U_inv @ A @ U
    Z = np.zeros_like(A, dtype=np.float)
    if k1 != 0:
        M_inv = np.linalg.inv(V[:k1,:k1])
        Z[:k1,:k1] = M_inv
    return U @ Z @ U_inv

In [46]:
# Testing

A = np.array([[1, 3, 0, 0],
     [0, 1, 3, 0],
     [0, 0, 1, 3],
     [0, 0, 0, 0,]])

Ad = drazin(A, 1e-4)
is_drazin(A, 1, Ad) # returns True

True

### Problem 3:

In [79]:
def effective_resistance(A):
    m, n = np.shape(A)
    degs = np.sum(A, axis=0) # degrees of columns
    D = np.diag(degs) # degree matrix
    L = D - A
    I_n = np.identity(n)
    
    R = np.zeros_like(L, dtype=np.float)
    
    for i in range(n):
        L_i = np.copy(L)
        L_i[:,i] = I_n[:,i]
        L_iD = drazin(L_i)
        R[:,i] = np.diagonal(L_iD)
        R[i,i] = 0
        
    return R


In [81]:
A = [[0, 0, 1, 1, 0, 0],
     [0, 0, 0, 1, 0, 0],
     [1, 0, 0, 1, 0, 1],
     [1, 1, 1, 0, 0, 1],
     [0, 0, 0, 0, 0, 1],
     [0, 0, 1, 1, 1, 0]]

A2 = [[0, 1],
      [1, 0]]

effective_resistance(A2)

array([[ 0.,  1.],
       [ 1.,  0.]])

### Problem 4:

In [ ]:
class LinkPredictor(object):
    
    def __init__(self, filename):
        names = pd.read_csv(filename)
        names = np.unique(names)
        